In [ ]:
import sympy as sm
import sympy.physics.mechanics as me
import numpy as np

from scipy.optimize import minimize, root
from scipy.integrate import solve_ivp
import itertools as itt
from matplotlib import animation
from IPython.display import HTML
import matplotlib as mp
import matplotlib
from matplotlib import patches
import matplotlib.pyplot as plt
%matplotlib inline
import time
import copy
matplotlib.rcParams['animation.embed_limit'] = 2**128

Needed to exit the loop when trying to place the ellipses, and when finding contact points during integration.

In [ ]:
class Rausspringen(Exception):
    pass

This creates a decorator to test functions for usage of CPU time, line by line.\
To see the results, this line: *profiler.print_stats()* must be added. 

In [ ]:
from line_profiler import LineProfiler

profiler = LineProfiler()

def profile(func):
    def inner(*args, **kwargs):
        profiler.add_function(func)
        profiler.enable_by_count()
        return func(*args, **kwargs)
    return inner


Homogenious ellipses of mass *m* and parameters *a, b* are dropped or thrown on an uneven street. A particle of mass $m_o$ may be attached anywhere within each ellipse.\
The street is a 'curve' in the X/Y plane, gravitation points in the negative Y - direction.\
The impact is modelled using the **Hunt-Crossley method**, details below.

I cannot model friction between the ellipse and the street (see sympy issue 25307).

**Parameters**
- *N*: inertial frame
- $A_{list}$: frames fixed to the ellipses
- $O$: point fixed in *N*
- $Dmc_{list}$: center of the ellipses
- $Po_{list}$: location of the particles fixed to the ellipses

- $q_{list}, u_{list}$: angle of rotation of the ellipses, its speed
- x_list, y_list, ux_list, uy_list*: coordinates of the center of the ellipse, its speeds

- $\epsilon_{s_{list}}$: contact angel for colission with the street
- *xs_list*: holds the X coordinates of the colission points on the street.
- *ls_list*: distance from ellipses to the street.
- *le_list*: distance from one ellipse to another one
- $\epsilon_{e_{list}}$ holds the colission angles of each pair of ellipses


- $m, m_o$: mass of the ellipse, of the particle attached to the ellipse
- *a, b*: semi axes of the ellipse
- *amplitude, frequenz*: parameters for the street.
- $i_{ZZ}$: moment of inertia of the ellipse around the Z axis
- $\alpha_{list}, \beta_{list}$: determine the location of the particel w.r.t. Dmc
- $\nu_s, \nu_e, EY_s, EY_e$: Poisson's ratio and Young's mudulus of street and ellipses respectively

In [ ]:
start = time.time()

#====================
n = 2
#====================
if n < 2 or isinstance(n, int) != True:
    raise Exception('n must be an integer > 1')

    
N = me.ReferenceFrame('N')
O = me.Point('O')
O.set_vel(N, 0)
t = me.dynamicsymbols._t

m, mo, g, a, b, reibung = sm.symbols('m, mo, g, a, b, reibung')
nue, nus, EYe, EYs, ctau = sm.symbols('nue, nus EYe, EYs, ctau')
amplitude, frequenz = sm.symbols('amplitude, frequenz')
x = sm.symbols('x')

q_list = list(me.dynamicsymbols(f'q:{n}'))
x_list = list(me.dynamicsymbols(f'x:{n}'))
y_list = list(me.dynamicsymbols(f'y:{n}'))

u_list  = list(me.dynamicsymbols(f'u:{n}'))
ux_list = list(me.dynamicsymbols(f'ux:{n}'))
uy_list = list(me.dynamicsymbols(f'uy:{n}'))

A_list     = list(sm.symbols(f'A:{n}', cls=me.ReferenceFrame))
Dmc_list   = list(sm.symbols(f'Dmc:{n}', cls=me.Point))
Po_list    = list(sm.symbols(f'Po:{n}', cls=me.Point))
CPhes_list = list(sm.symbols(f'CPhes:{n}', cls= me.Point))                 # contact point on the ellipse w.r.t. the street
CPhss_list = list(sm.symbols(f'CPhss:{n}', cls=me.Point))                  # "Partner point" on the street
CPhee_list = list(sm.symbols(f'CPhee:{n}', cls=me.Point))                   # Contact point of ellipse when colliding with another ellipse

rhodts_list   = list(sm.symbols(f'rhodts:{n}'))  
xs_list       = list(sm.symbols(f'xs:{n}'))
ls_list       = list(sm.symbols(f'ls:{n}'))
epsilons_list = list(sm.symbols(f'epsilons:{n}'))
alpha_list    = list(sm.symbols(f'alpha:{n}'))
beta_list     = list(sm.symbols(f'beta:{n}'))

rhodte_list   = [sm.symbols(f'rhodte{i}{j}') for i, j in itt.permutations(range(n), r=2)]
le_list       = [sm.symbols(f'le{i}{j}') for i, j in itt.permutations(range(n), r=2)]
epsilone_list = [sm.symbols(f'epsilone{i}{j}, epsilone{j}{i}') for i, j in itt.permutations(range(n), r=2)]
richtung_list = [sm.symbols(f'richtungx{i}{j}, richtungy{i}{j}')for i, j in itt.permutations(range(n), r=2)]

Body1 = []
Body2 = []
iZZ   = 0.25 * m * (a**2 + b**2)  # moment of inertia of the ellipse w.r.t its center, normal to its plane

for i in range(n):
    A_list[i].orient_axis(N, q_list[i], N.z)
    A_list[i].set_ang_vel(N, u_list[i] * N.z)

    Dmc_list[i].set_pos(O, x_list[i]*N.x + y_list[i]*N.y)
    Dmc_list[i].set_vel(N, ux_list[i]*N.x + uy_list[i]*N.y)

    CPhes_list[i].set_pos(Dmc_list[i], a*sm.cos(epsilons_list[i])*A_list[i].x + b*sm.sin(epsilons_list[i])*A_list[i].y)
    CPhes_list[i].v2pt_theory(Dmc_list[i], N, A_list[i])

    Po_list[i].set_pos(Dmc_list[i], a*alpha_list[i]*A_list[i].x + b*beta_list[i]*A_list[i].y)
    Po_list[i].v2pt_theory(Dmc_list[i], N, A_list[i])

    I = me.inertia(A_list[i], 0, 0, iZZ)                                              
    body = me.RigidBody('body' + str(i), Dmc_list[i], A_list[i], m, (I, Dmc_list[i]))
    teil = me.Particle('teil' + str(i), Po_list[i], mo)
    Body1.append(body)
    Body2.append(teil)

BODY = Body1 + Body2

Model the street.\
It is a parabola, open to the top, with superimposed sinus waves.\
Then I calculate the formula for its osculating circle, the formula of which I found in the internet.

In [ ]:
#Modeling the street
#============================================
rumpel = 5  # the higher the number the more 'uneven the street'
#============================================
def gesamt1(x, amplitude, frequenz):
    strasse = sum([amplitude/j * sm.sin(j*frequenz * x) for j in range(1, rumpel)])
    strassen_form = (frequenz/2. * x)**2
    gesamt = strassen_form + strasse
    return gesamt

r_max = ((sm.S(1.) + (gesamt1(x, amplitude, frequenz).diff(x))**2 )**sm.S(3/2)/
         gesamt1(x, amplitude, frequenz).diff(x, 2))

**Find the point where the ellipse hits the street**\
The idea is as follows:\
When the ellipse hits the street, the tangent at the ellipse at the hitting point, and the tangent at the street at the hitting point must be parallel. So, I look for the point where the ellipse would touch the street if it was inflated to touch the street for every point of the integration time. This sequence of potential hitting points will eventually give the real hitting point.
- let $CP_{hs}$ be the point of the street where a multiple of the vector $\hat n$ which is normal to the tanget of the ellipse at $CP_h \in$ *circumference of ellipse* intersects with the street below
-  is the tangent of the ellipse at $CP_h$ parallel to the tangent of the street at $CP_{hs}$?
- if YES, $CP_{hs}$ is a potential impact point.
- collect all potential impact points, and select the one closest to the ellipse. This is the point the ellipse would touch, if it were 'blown up' to just touch the street.

All this has to be done numerically during integration, I could not come up with another way.

In more **detail**:\
To get the derivative of the ellipse at the point $(x, y) \in $ {circumference of ellipse} calculate: $\dfrac{d}{dx}(\dfrac{x^2}{a^2}\space + \space \dfrac{y^2}{b^2} \space = \space 1.)$ to get:\
$\dfrac{dy}{dx} = - \dfrac{b^2}{a^2} \cdot \dfrac{x}{y} \space$ for $y \neq 0$\
hence the normalized tanget vector is\
$t_{ellipse} = (\hat{ A.x + \dfrac{dy}{dx} \cdot A.y}) \space$ for $\space y \neq 0$\
$t_{ellipse}$ = +/-$A.y \space \space \space \space \space  \space \space \space \space \space \space \space \space \space \space \space \space \space \space \space$ for $\space   y = 0.$

Therefore the normal vector is:\
$\hat n = (\hat{ \dfrac{dy}{dx} \cdot A.x - A.y}) \space$ for $\space y \neq 0$\

$\hat n = \space A.x \space \space \space \space \space \space \space \space \space \space \space \space \space \space \space \space \space \space \space \space$ for $\space   y = 0., x = a$

$\hat n$ = -$A.x \space \space \space \space \space \space \space \space \space \space \space \space \space \space \space \space \space \space \space \space$ for $\space   y = 0., x = -a$

Find the **location** of $CP_{hs}$:\
$CP_{hs} \in gesamt(x, parameters), \space$ where the function *gesamt(x, parameters)* models the street.

Let $l = |{}^{CP_{hd}} r^{CP_h}| = |l \hat n|$, then we get two equations:

$(l\hat n \cdot N.x) = x$\
$(l\hat n \cdot N.y) = gesamt(x), \space$

to be solved during each step of the numerical integration.\
I solve it only if  $ \hat n \cdot N.y \leq 0.$ For the shape of my street, this seems adequate and saves integration time.

I take $\sin(\theta) = | \space tanget_{ellipse} \times tangent_{street} \space |$  $\space$ as a measure how 'parallel' the tangents at the collision points are, where the tangents are unit vectors, and $\theta \space$ is the angle between them.

**NOTE**\
Initially I tried to get the distance from an ellipse to the street similarly as I do it with the ellipses: Set up the function *distance(ellipse to street)* = $f(\epsilon, x, parameters)$, where $\epsilon$ gives a point on the circumference of the ellipse, and x gives a point on the street, and minimize it. This did not work for me for two reasons:
- using scipy's *minimize()* directly was very slow, and probably too inacurate for solve_ivp to get a result. It would terminate with an error message like *rerquired accuracy cannot be achieved* or similar.
- solving $\nabla_{\epsilon, x} f(\epsilon, x, parameters) = 0$. This is only a necessary condition. When the osculating radius of the street is smaller than the one of the ellipses, the coordinates of the impact point of ellipse and streetmay depend *discontinuously* of the generalized coordinates of the ellipse. Hence the *old* location of the minimal distance may not be a good initial guess for the next location, and the solution may converge to some other local extreme value of the distance function. Whatever the reasons, I never got it to work.\
(This does not happen between collissions of ellipse with each other - and there it worked fine, see below) 


In [ ]:
def equation_street(i, delta):
# This function returns the equations to be solven for l and for x, as explained above.
#find the vector normal to the tanget at the unrotated ellipse at the point CPh
    CPhx = a * sm.cos(delta)
    CPhy = b * sm.sin(delta)
    ausdruckk = (sm.Abs(CPhy) <= 1.e-15)
    ausdruckg = (sm.Abs(CPhy) >  1.e-15)

    dydx = sm.Piecewise((-b**2/a**2 * CPhx/CPhy, ausdruckg), (1.e15, ausdruckk), (1., True))

    hilfsx = sm.Piecewise((1., delta == sm.S(0)), (-1., delta == sm.pi ), (-dydx, delta < sm.pi/2.), (-dydx, delta < sm.pi), (dydx, delta < 3./2.*sm.pi), (dydx, delta < 2.*sm.pi), (1., True) )
    hilfsy = sm.Piecewise((0., ausdruckk),                                                       (1., delta < sm.pi/2.),    (1., delta < sm.pi), (-1.,  delta < 3./2.*sm.pi), (-1,     delta < 2.*sm.pi), (1., True) )

    nhat = (hilfsx*A_list[i].x + hilfsy*A_list[i].y).normalize()
#print('nhat DS',me.find_dynamicsymbols(nhat, reference_frame=N))
#print('nhat FS', nhat.free_symbols(reference_frame=N))
    CPhss_list[i].set_pos(CPhes_list[i], ls_list[i]*nhat)
    hilfs1 = CPhss_list[i].pos_from(O)

# CPhs_ort to be solved for l and for x numerically later duering each step of the integration.
    CPhs_ort = sm.Matrix([me.dot(hilfs1, N.x) - xs_list[i], me.dot(hilfs1, N.y) - gesamt1(xs_list[i], amplitude, frequenz)])
#    print('CPhs_ort DS', me.find_dynamicsymbols(CPhs_ort))
#    print('CPhs_ort FS', CPhs_ort.free_symbols)
#    print(f'CPhs_ort has {sm.count_ops(CPhs_ort)} operations. After cse it has {sm.count_ops(sm.cse(CPhs_ort))}')
    return CPhs_ort       # nhat, that are retuned for use in other functions only

def nhat_that(i, delta):
# This function returns the equations to be solven for l and for x, as explained above.
#find the vector normal to the tanget at the unrotated ellipse at the point CPh
    CPhx = a * sm.cos(delta)
    CPhy = b * sm.sin(delta)
    ausdruckk = (sm.Abs(CPhy) <= 1.e-15)
    ausdruckg = (sm.Abs(CPhy) >  1.e-15)

    dydx = sm.Piecewise((-b**2/a**2 * CPhx/CPhy, ausdruckg), (1.e15, ausdruckk), (1., True))

    hilfsx = sm.Piecewise((1., delta == sm.S(0)), (-1., delta == sm.pi ), (-dydx, delta < sm.pi/2.), (-dydx, delta < sm.pi), (dydx, delta < 3./2.*sm.pi), (dydx, delta < 2.*sm.pi), (1., True) )
    hilfsy = sm.Piecewise((0., ausdruckk),                                                       (1., delta < sm.pi/2.),    (1., delta < sm.pi), (-1.,  delta < 3./2.*sm.pi), (-1,     delta < 2.*sm.pi), (1., True) )
    nhat = (hilfsx*A_list[i].x + hilfsy*A_list[i].y).normalize()
    that = (A_list[i].x + dydx*A_list[i].y).normalize()
    return [nhat, that]



def parallel_street(i):
# this function returns the sin of the deviation of the tangents of ellipse and street from being parallel, see above
    strasse = gesamt1(x, amplitude, frequenz)
    strassedx = strasse.diff(x).subs({x: xs_list[i]})
    tangente_strasse = (N.x + strassedx *N.y).normalize()
    that = nhat_that(i, epsilons_list[i])[1]
    parallel = (that.cross(tangente_strasse)).magnitude()
    return parallel

equation_street_lam       = []
equation_street_jakob_lam = []
for i in range(n):
    CPhs_ort1 = equation_street(i, epsilons_list[i])
    jakob     = CPhs_ort1.jacobian([xs_list[i], ls_list[i]])
    print('equation FS', CPhs_ort1.free_symbols)
    print('equation DS', me.find_dynamicsymbols(CPhs_ort1)) 
    equation_street_lam.append(sm.lambdify([xs_list[i], ls_list[i]] + q_list + x_list + y_list + [xs_list[j] for j in range(n) if j !=i] + [ls_list[j] for j in range(n) if j !=i] + epsilons_list + [a, b, amplitude, frequenz], CPhs_ort1, cse=True))
    equation_street_jakob_lam.append(sm.lambdify([xs_list[i], ls_list[i]] + q_list + x_list + y_list + [xs_list[j] for j in range(n) if j !=i] + [ls_list[j] for j in range(n) if j !=i] + epsilons_list + [a, b, amplitude, frequenz], jakob, cse=True))


parallel_street_lam = []
for i in range(n):
    p1 = parallel_street(i)
    print('parallel Fs', p1.free_symbols)
    print('parallel DS', me.find_dynamicsymbols(p1)) 
    parallel_street_lam.append(sm.lambdify(q_list + xs_list + epsilons_list + ls_list + [a, b, amplitude, frequenz], p1, cse=True))

# This is needed only for the plot with the initial conditions
liste_lam     = []
nhat_lam      = []
senkrecht_lam = []
for i in range(n):
    CPhx = a * sm.cos(epsilons_list[i])
    CPhy = b * sm.sin(epsilons_list[i])
    CPha = me.Point('CPha')
    CPhe = me.Point('CPhe')
    CPha.set_pos(Dmc_list[i], CPhx*A_list[i].x + CPhy*A_list[i].y)
    nhat = nhat_that(i, epsilons_list[i])[0]
    CPhe.set_pos(CPha, 1.*nhat)
    liste = [[me.dot(punkt.pos_from(O), uv) for uv in (N.x, N.y)] for punkt in (CPha, CPhe)]
    liste_lam.append(sm.lambdify(q_list + x_list + y_list + epsilons_list + [a, b], liste, cse=True))

    nhat = nhat_that(i, epsilons_list[i])[0]

    print('nhat FS', me.dot(nhat, N.x).free_symbols)
    print('nhat DS', me.find_dynamicsymbols(me.dot(nhat, N.x))) 
    nhat_lam.append(sm.lambdify(q_list + epsilons_list + [a, b], [me.dot(nhat, uv) for uv in (N.x, N.y)]))

    that, nhat = nhat_that(i, epsilons_list[i])[0], nhat_that(i, epsilons_list[i])[1]
    senkrecht = me.dot(nhat, that)
    senkrecht_lam.append(sm.lambdify(q_list + epsilons_list + [a, b], senkrecht, cse=True))

**Find the potential collision points of any two ellipses**\
I assume, that no more than two ellipses start a collision at the same time. Since the initial conditons are random, I believe that Prob(more than two ellipses start to collide at the same time) $\approx 0.$\
In order to get the potential contact points, say, $CPhe_i, CPhe_j$, I simply try to find the minimum distance between any two points on the circumferences on the respective ellipses.\
I do this in two ways:
- minimize $|{}^{CPhe_i} \bar r^{CPhe_j}(\epsilon_i, \epsilon_j)|$ directly, using scipy's minimize function
- calculate $\dfrac{d}{d\epsilon_i} |{}^{CPhe_i} \bar r^{CPhe_j}(\epsilon_i, \epsilon_j)|$ and $\dfrac{d}{d\epsilon_j} |{}^{CPhe_i} \bar r^{CPhe_j}(\epsilon_i, \epsilon_j)|$, and solve for $\epsilon_i, \epsilon_j$. This is only a sufficient condition for a minimum, it could also give a maximum. With the right initial guess, scipy's **root** function should give the minimum. During integration, this seems to be faster than the first option. 


If $| {}^{CPhe_i} \bar r^{CPhe_j} | \approx 0.$ it becomes numerically critical to get the direction ${}^{CPhe_i} \bar r^{CPhe_j}$. So, I fix the direction before the distance becomes too small. See also the comment in the numerical integration.

In [ ]:
def CPhxe(epsilon):
    return a * sm.cos(epsilon)
def CPhye(epsilon):
    return b * sm.sin(epsilon)

def vorzeichen(i, j, epsilon1, epsilon2):
# the idea is this: I calculate the triangle abc, with a := Dmc_i - CPh_i, b := CPhe_j, and get c using
# the cosine theorem: c^2 = a^2 + b^2 - 2 * a * b * cos(gamma)
# while c < Dmc_i.pos_from(Dmc_j).magnitude, the ellipses are separated.
    Pi = me.Point('Pi')
    Pj = me.Point('Pj')
    Pi.set_pos(Dmc_list[i], CPhxe(epsilon1)*A_list[i].x + CPhye(epsilon1)*A_list[i].y)
    Pj.set_pos(Dmc_list[j], CPhxe(epsilon2)*A_list[j].x + CPhye(epsilon2)*A_list[j].y)

    rr = Dmc_list[i].pos_from(Dmc_list[j]).magnitude()
    r1 = Dmc_list[i].pos_from(Pi)
    r2 = Dmc_list[j].pos_from(Pj)
    gamma_cos = (me.dot(r1.normalize(), r2.normalize()))
    r1 = r1.magnitude()
    r2 = r2.magnitude() 
    r3 = sm.sqrt(r1**2 + r2**2 - 2. * r1 * r2 * gamma_cos)
    hilfs1 = rr - r3
    hilfs2 = sm.Piecewise((-1., hilfs1 <= 0.), (1., hilfs1 > 0.))
    return hilfs2  # -1., if the ellipses have penetrated


def distanzCPheiCPhej(i, j, epsilon1, epsilon2):
    P1, P2 = sm.symbols('P1, P2', cls=me.Point)
    P1.set_pos(Dmc_list[i], CPhxe(epsilon1)*A_list[i].x + CPhye(epsilon1)*A_list[i].y)
    P2.set_pos(Dmc_list[j], CPhxe(epsilon2)*A_list[j].x + CPhye(epsilon2)*A_list[j].y)
    vektor = P2.pos_from(P1)
    return vektor.magnitude() * vorzeichen(i, j, epsilon1, epsilon2)    

def richtungCPheiCPhej(i, j, epsilon1, epsilon2):
    P1, P2 = sm.symbols('P1, P2', cls=me.Point)
    P1.set_pos(Dmc_list[i], CPhxe(epsilon1)*A_list[i].x + CPhye(epsilon1)*A_list[i].y)
    P2.set_pos(Dmc_list[j], CPhxe(epsilon2)*A_list[j].x + CPhye(epsilon2)*A_list[j].y)
    vektor = P2.pos_from(P1).normalize()
    return [me.dot(vektor, uv) for uv in (N.x, N.y) ]

# This function will be minimized numerically during integratrion to get the distance from ellipse_i to ellipse_j
# ...._lam for the direct minimization, ...lam1 for the gradient method.
min_distanzCPheiCPhej_lam  = []
min_distanzCPheiCPhej_lam1 = []
jakob_lam = []
jakob_lam1 = []

zaehler = -1
for i, j in itt.permutations(range(n), r=2):
    zaehler += 1
    abstand = distanzCPheiCPhej(i, j, epsilone_list[zaehler][0], epsilone_list[zaehler][1])
    abstanddeidej = [abstand.diff(epsilone_list[zaehler][0]), abstand.diff(epsilone_list[zaehler][1])]
    jakob = sm.Matrix([abstand.diff(epsilone_list[zaehler][0]), abstand.diff(epsilone_list[zaehler][1])])
    
    hilfs1 = sm.Matrix([abstand.diff(epsilone_list[zaehler][0]), abstand.diff(epsilone_list[zaehler][1])])
    jakob1 = hilfs1.jacobian([epsilone_list[zaehler][0], epsilone_list[zaehler][1]])

    min_distanzCPheiCPhej_lam.append((sm.lambdify([epsilone_list[zaehler][0], epsilone_list[zaehler][1]] + q_list + x_list + y_list + [a, b], abstand, cse=True)))
    min_distanzCPheiCPhej_lam1.append(sm.lambdify([epsilone_list[zaehler][0], epsilone_list[zaehler][1]] + q_list + x_list + y_list + [a, b], abstanddeidej, cse=True))

    jakob_lam1.append(sm.lambdify([epsilone_list[zaehler][0], epsilone_list[zaehler][1]] + q_list + x_list + y_list + [a, b], jakob1, cse=True))
    jakob_lam.append(sm.lambdify([epsilone_list[zaehler][0], epsilone_list[zaehler][1]] + q_list + x_list + y_list + [a, b], jakob, cse=True))



richtung_lam = []
zaehler = -1
for i, j in itt.permutations(range(n), r=2):
    zaehler +=1
    richtungij = richtungCPheiCPhej(i, j, epsilone_list[zaehler][0], epsilone_list[zaehler][1])
    richtung_lam.append((sm.lambdify([epsilone_list[zaehler][0], epsilone_list[zaehler][1]] + q_list + x_list + y_list + [a, b], richtungij, cse=True)))


# just needed for plotting initial situation
epsilonei = sm.symbols('epsilonei')
CPhe = list(sm.symbols('CPhe' + str(i), cls=me.Point) for i in range(n))

CPhe_list = []
for i in range(n):
    CPhe[i].set_pos(Dmc_list[i], CPhxe(epsilonei)*A_list[i].x + CPhye(epsilonei)*A_list[i].y )
    CPhe_list.append([me.dot(CPhe[i].pos_from(O), uv) for uv in (N.x, N.y)])
CPhe_list_lam = sm.lambdify(q_list + x_list + y_list + [a, b, epsilonei], CPhe_list, cse=True)

**Calculate the impact speeds**

In [ ]:
speed_dict = {sm.Derivative(i, t): j for i, j in zip(q_list + x_list + y_list, u_list + ux_list + uy_list)}
def equation_rhodts(i, delta):
# returns the speed component of the potential contact point on the ellipse in the direction of the 'impact line'    
    vs = me.msubs(CPhes_list[i].pos_from(O).diff(t, N), speed_dict)
    vs = me.dot(vs, nhat_that(i, delta)[0])
    return vs

def equation_rhodte(i, j, deltai, deltaj, richtungx, richtungy):
# returns the speed component of the difference between ellipse_i and ellipse_j, in direction of their impact line.
    P1, P2 = sm.symbols('P1, P2', cls=me.Point)
    P1.set_pos(Dmc_list[i], a*sm.cos(deltai)*A_list[i].x + b*sm.cos(deltai)*A_list[i].y)
    P2.set_pos(Dmc_list[j], a*sm.cos(deltaj)*A_list[j].x + b*sm.cos(deltaj)*A_list[j].y)

    vs1 = me.msubs(P1.pos_from(O).diff(t, N), speed_dict)
    vs2 = me.msubs(P2.pos_from(O).diff(t, N), speed_dict)
    speed = me.dot(vs2 - vs1, richtungx*N.x + richtungy*N.y)
    return speed
    

rhodts_lam = []
for i in range(n):
    speed = equation_rhodts(i, epsilons_list[i])
    print('rhodts FS', speed.free_symbols)
    print('rhodts DS', me.find_dynamicsymbols(speed)) 
    rhodts_lam.append(sm.lambdify(q_list + x_list + y_list + u_list + ux_list + uy_list + epsilons_list + [a, b, amplitude, frequenz], speed, cse=True))

rhodte_lam = []
zaehler = -1
for i, j in itt.permutations(range(n), r=2):
    zaehler += 1
    speed = equation_rhodte(i, j, epsilone_list[zaehler][0], epsilone_list[zaehler][1], richtung_list[zaehler][0], richtung_list[zaehler][1])
    print('rhodte FS', speed.free_symbols)
    print('rhodte DS', me.find_dynamicsymbols(speed)) 
    rhodte_lam.append(sm.lambdify(q_list + x_list + y_list + u_list + ux_list + uy_list + epsilone_list + richtung_list + [a, b], speed, cse=True))

**Force on $CP_h$ during impact of an ellipse with the street**

**Force acting on $CP_h$ during impact**\
I use Hunt_Crossley's method to calculate it.

*Hunt Crossley's method*
 
My reference is this article, given to me by JM\
https://www.sciencedirect.com/science/article/pii/S0094114X23000782 \

 
This is with dissipation during the collision, the general force is given in (63) as\
$f_n = k_0 \cdot \rho + \chi \cdot \dot \rho$, with $k_0$ as above, $\rho$ the penetration, and $\dot\rho$ the speed of the penetration.\
In the article it is stated, that $n = \frac{3}{2}$ is a good choice, it is derived in Hertz' approach. Of course, $\rho, \dot\rho$ must be the signed magnitudes of the respective vectors.

A more realistic force is given in (64) as:\
$f_n = k_0 \cdot \rho^n + \chi \cdot \rho^n\cdot \dot \rho$, as this avoids discontinuity at the moment of impact.

**Hunt and Crossley** give this value for $\chi$, see table 1:

$\chi = \dfrac{3}{2} \cdot(1 - c_\tau) \cdot \dfrac{k_0}{\dot \rho^{(-)}}$, 
where $c_\tau = \dfrac{v_1^{(+)} - v_2^{(+)}}{v_1^{(-)} - v_2^{(-)}}$, where $v_i^{(-)}, v_i^{(+)}$ are the speeds of $body_i$, before and after the collosion, see (45), $\dot\rho^{(-)}$ is the speed right at the time the impact starts. $c_\tau$ is an experimental factor, apparently around 0.8 for steel.

Using (64), this results in their expression for the force:

$f_n = k_0 \cdot \rho^n \left[1 + \dfrac{3}{2} \cdot(1 - c_\tau) \cdot \dfrac{\dot\rho}{\dot\rho^{(-)}}\right]$

with $k_0 = \frac{4}{3\cdot(\sigma_1 + \sigma_2)} \cdot \sqrt{\frac{R_1 \cdot R_2}{R_1 + R_2}}$, where $\sigma_i = \frac{1 - \nu_i^2}{E_i}$, with $\nu_i$ = Poisson's ratio, $E_i$ = Young"s modulus, $R_1, R_2$ the radii of the colliding bodies, $\rho$ the penetration depth. All is near equations (54) and (61) of this article.

1. Penetration depth $rho$:\
From the description in the cell above, it is clear that\
$rho = |l| \cdot H(-l) \space$, with l from above (found numerically), and $H(...)$ being the heaviside function.


2. Determine $R_1, R_2$ in the above formulas:\
For a function $y = f(x)$ the signed curvature is:\
$\kappa = \dfrac{\frac{d^2}{dx^2} f(x)}{(1 + (\frac{d}{dx} f(x))^2)^{\frac{3}{2}} }$\
For an ellipse, $\kappa = \dfrac{a \cdot b}{ \left( \sqrt{a^2 \sin^2(\delta) + b^2 \cos^2(\delta)} \right)^3}  > 0 \space \space \forall \delta \in [0, 2 \pi), \space$ where $\delta$ is the angle from A.x to the point.\
As an approximation for $R_1, R_2$ I take the radius of the osculating circle, which is:\
$R_i = \dfrac{1}{\kappa_i} \space$ If the penetration depth is no too large, this should be o.k.\
Note, that I allow *negative* $R_2$: This means, the street is concave from the ellipse's point of view.\
At a contact point, either $R_2 > 0$ or $|R_2| \leq R_1$ there should be no problems.\
I do not know, whether this approach is within the **validity of the H-C method**


3. Penetration speed $\frac{d}{dt} \rho(t)$\
Only the component of $\frac{d}{dt} CP_h(t)$ is relevant, hence:\
$\frac{d}{dt} \rho(t) = \frac{d}{dt} CP_h \cdot \hat n$

*spring energy* =   $ k_0 \cdot \int_{0}^{\rho} k^{3/2}\,dk$ = $k_0 \cdot\frac{2}{5} \cdot \rho^{5/2}$\
I assume, the dissipated energy cannot be given in closed form, at least the article does not give one.

*Note*  
$c_\tau = 1.$ gives **Hertz's** solution to the impact problem, also described in the article.


**Friction when the ellipse hits the street**\
This cannot be done, seems it will result in too many *Piecewise(...)* terms, see sympy issue #25307, hence I do not show the simple code.

In [ ]:
def impact_force_street(i, delta, rhodts_list):
# impact force on CPh
# curvature of the ellipse at the point (a*cos(delta) / b*sin(delta)) from the internet
    kappa1 = (a * b) / (sm.sqrt((a*sm.sin(delta))**2 + (b*sm.cos(delta))**2))**3

# formula for the curvature of a function. From the internet.
    hilfs = gesamt1(x, amplitude, frequenz)
    hilfsdx = hilfs.diff(x).subs({x: xs_list[i]})
    hilfsdxdx = hilfs.diff(x, 2).subs({x: xs_list[i]})
    kappa2 = sm.Piecewise((hilfsdxdx**2 / (1. + hilfsdx**2)**1.5, hilfsdxdx != 0.), (1.e-15, True))

    R1 = 1. / kappa1
    R2 = 1. / kappa2
    sigmae = (1. - nue**2) / EYe
    sigmas = (1. - nus**2) / EYs
    k0 = 4./3. * 1./(sigmae + sigmas) * sm.sqrt(R1*R2 / (R1 + R2))
 
    rhodt = equation_rhodts(i, delta)
    l = ls_list[i]
    rho   = sm.Abs(l) * sm.Heaviside(-l, sm.S(0))
#print('rhodt DS', me.find_dynamicsymbols(rhodt))
#print('rhodt FS', rhodt.free_symbols)
    nhat = nhat_that(i, delta)[0]
    fHC_betrag = k0 * rho**(3/2) * (1. + 3./2. * (1. - ctau) * (rhodt) / sm.Abs(rhodts_list[i])) 
    fHC = fHC_betrag * (-nhat) * sm.Heaviside(-l, sm.S(0))  # force is acting on CPhes hence the minus sign.
#    print('fHC DS', me.find_dynamicsymbols(fHC, reference_frame=N))
#    print('fHC FS', fHC.free_symbols(reference_frame=N))

    return fHC

#print('fHC DS', me.find_dynamicsymbols(fHC, reference_frame=N))
#print('fHC FS', fHC.free_symbols(reference_frame=N))

**Collision between any two ellipses**\
I use Hunt-Crossley's method to model it.

*Friction when the ellipse hits another ellipse*\
This cannot be done, seems it will result in too many *Piecewise(...)* terms, see sympy issue #25307, hence I do not show the simple code.

In [ ]:
#@profile
def impact_force_ellipse(i, j, epsiloni, epsilonj, l, rhodtellipse, richtungx, richtungy):
# this calculates the force of ellipse_i on ellipse_j during their collision.
# i, j are the respective ellipses
# epsilone list of angles
# l is the distance between ellipse_i and ellipse_j, negative during penetration
# rhodtellipse is the collision speed right before impact.
    
# curvature of the ellipse at the point (a*cos(delta) / b*sin(delta)) from the internet
    kappa1 = (a * b) / (sm.sqrt((a*sm.sin(epsiloni))**2 + (b*sm.cos(epsiloni))**2))**3
    kappa2 = (a * b) / (sm.sqrt((a*sm.sin(epsilonj))**2 + (b*sm.cos(epsilonj))**2))**3

    R1 = 1. / kappa1
    R2 = 1. / kappa2
    sigmae = (1. - nue**2) / EYe
    k0 = 4./3. * 1./(sigmae + sigmae) * sm.sqrt(R1*R2 / (R1 + R2))

    P1, P2 = sm.symbols('P1, P2', cls=me.Point)
    P1.set_pos(Dmc_list[i], CPhxe(epsiloni)*A_list[i].x + CPhye(epsiloni)*A_list[i].y)
    P2.set_pos(Dmc_list[j], CPhxe(epsilonj)*A_list[j].x + CPhye(epsilonj)*A_list[j].y)
    
    
    vei = P1.pos_from(O).diff(t, N)
    vej = P2.pos_from(O ).diff(t, N)
# only the speed in direction of the collision is important here
    richtung = richtungx*N.x + richtungy*N.y                            # points from ellipse_i to ellipse_j
    rhodt = me.msubs(me.dot(vei - vej, richtung), speed_dict)
    rho   = sm.Abs(l) * sm.Heaviside(-l, sm.S(0))
#    print('rhodt DS', me.find_dynamicsymbols(rhodt))
#    print('rhodt FS', rhodt.free_symbols)

    fHC_betrag = k0 * rho**(3/2) * (1. + 3./2. * (1. - ctau) * (rhodt) / sm.Abs(rhodtellipse))
    fHC = fHC_betrag * richtung * sm.Heaviside(-l, sm.S(0))
    return fHC

**Kane's equations**\
There is nothing special here.

In [ ]:
start1 = time.time()

FL_gravity = [(Dmc_list[i], -m*g*N.y) for i in range(n)] + [(Po_list[i], -mo*g*N.y) for i in range(n)]

FL_impact_street  = []
for i in range(n):
    FL_impact_street.append((CPhes_list[i], impact_force_street(i, epsilons_list[i], rhodts_list)))

CPhejj = list(sm.symbols(f'CPhejj:{n}', cls =me.Point))
FL_impact_ellipse = []
zaehler = -1
for i, j in itt.permutations(range(n), r=2):
    zaehler += 1    
    CPhejj[j].set_pos(Dmc_list[j], a*sm.cos(epsilone_list[zaehler][1])*A_list[j].x + b*sm.sin(epsilone_list[zaehler][1])*A_list[j].y)
    CPhejj[j].set_vel(N, CPhejj[j].pos_from(O).diff(t, N))
    FL_impact_ellipse.append((CPhejj[j], impact_force_ellipse(i, j, epsilone_list[zaehler][0], epsilone_list[zaehler][1], le_list[zaehler], rhodte_list[zaehler], richtung_list[zaehler][0], richtung_list[zaehler][1])))

FL = FL_gravity + FL_impact_street + FL_impact_ellipse

kd = [i - j.diff(t) for i, j in zip((u_list + ux_list + uy_list), (q_list + x_list + y_list))]

q_ind = q_list + x_list + y_list
u_ind = u_list + ux_list + uy_list

#@profile
def KANE():
    for _ in range(1):
        KM = me.KanesMethod(N, q_ind=q_ind, u_ind=u_ind, kd_eqs=kd) 
        (fr, frstar) = KM.kanes_equations(BODY, FL)
        MM = KM.mass_matrix_full
        force = KM.forcing_full
    return MM, force
MM, force = KANE()

print('force DS', me.find_dynamicsymbols(force))
print('force free symbols', force.free_symbols)
print(f'force has {sm.count_ops(force)} operations, {sm.count_ops(sm.cse(force))} operations after cse', '\n')

print('MM DS', me.find_dynamicsymbols(MM))
print('MM free symbols', MM.free_symbols)
print(f'MM has {sm.count_ops(MM)} operations, {sm.count_ops(sm.cse(MM))} operations after cse', '\n')

print(f'it took {time.time() - start1 :.5f} sec to establish Kanes equations')

Here the *sympy functions* are converted to *numpy functions* so numerical calculations may be done.\
Before, I calculate the functions for the *potential energy*, for the *kinetic energy* and for the *spring energy*.

In [ ]:
start1 = time.time()
# ENERGIES
pot_energie = sum([m * g * me.dot(punkt.pos_from(O), N.y) for punkt in Dmc_list]) + sum([mo * g * me.dot(punkt.pos_from(O), N.y) for punkt in Po_list])
kin_energie = sum([koerper.kinetic_energy(N) for koerper in BODY])

spring_energie = 0.
# collision between ellipose and stree
for i in range(n):
      kappa1 = (a * b) / (sm.sqrt((a*sm.sin(epsilons_list[i]))**2 + (b*sm.cos(epsilons_list[i]))**2))**3
# formula for the curvature of a function. From the internet.
      hilfs = gesamt1(x, amplitude, frequenz)
      hilfsdx = hilfs.diff(x).subs({x: xs_list[i]})
      hilfsdxdx = hilfs.diff(x, 2).subs({x: xs_list[i]})
      kappa2 = sm.Piecewise((hilfsdxdx**2 / (1. + hilfsdx**2)**1.5, hilfsdxdx != 0.), (1.e-15, True))

      R1 = 1. / kappa1
      R2 = 1. / kappa2
      sigmae = (1. - nue**2) / EYe
      sigmas = (1. - nus**2) / EYs
      k0 = 4./3. * 1./(sigmae + sigmas) * sm.sqrt(R1*R2 / (R1 + R2))
      spring_energie += 2./5. * k0 * sm.Abs(ls_list[i])**(5/2) * sm.Heaviside(-ls_list[i], 0.)

# collisions between ellipses
zaehler = -1
for i, j in itt.permutations(range(n), r=2):
      zaehler += 1
      kappa1 = (a * b) / (sm.sqrt((a*sm.sin(epsilone_list[zaehler][0]))**2 + (b*sm.cos(epsilone_list[zaehler][0]))**2))**3
      kappa2 = (a * b) / (sm.sqrt((a*sm.sin(epsilone_list[zaehler][1]))**2 + (b*sm.cos(epsilone_list[zaehler][1]))**2))**3

      R1 = 1. / kappa1
      R2 = 1. / kappa2
      sigmae = (1. - nue**2) / EYe
      k0 = 4./3. * 1./(sigmae + sigmae) * sm.sqrt(R1*R2 / (R1 + R2))

      rho = sm.Abs(le_list[zaehler])
      rho = rho**(5/2)
      spring_energie += (k0 * 2./5. * rho * sm.Heaviside(-le_list[zaehler], 0.))  * 0.5               # factor 0.5 needed, because I count it twice


#-------------------------------------------------------------------------------------------------------------------------------------------------------------------
#-------------------------------------------------------------------------------------------------------------------------------------------------------------------
# LAMBDIFICATION
qL = q_ind + u_ind
pL = [m, mo, g, a, b, amplitude, frequenz] + [ctau, EYe, EYs, nue, nus] + xs_list + epsilons_list + ls_list + alpha_list + beta_list + rhodts_list + epsilone_list + le_list + rhodte_list + richtung_list

MM_lam    = sm.lambdify(qL + pL, MM, cse=True)
force_lam = sm.lambdify(qL + pL, force, cse=True)

gesamt     = gesamt1(x, amplitude, frequenz)
gesamt_lam = sm.lambdify([x, amplitude, frequenz], gesamt, cse=True)

Po_ort_lam = []
for i in range(n):
      Po_ort_lam.append(sm.lambdify(q_list + x_list + y_list + [a, b] + alpha_list + beta_list, [me.dot(Po_list[i].pos_from(O), uv) for uv in (N.x, N.y)], cse=True ))

pot_lam    = sm.lambdify(qL + pL, pot_energie, cse=True)
kin_lam    = sm.lambdify(qL + pL, kin_energie, cse=True)
spring_lam = sm.lambdify(qL + pL, spring_energie, cse=True)

r_max_lam = sm.lambdify([x, amplitude, frequenz], r_max, cse=True)
#k0_lam = sm.lambdify(qL + pL, k0*sm.Heaviside(-l, 0), cse = True)

# for the initial conditions only
Dmc_distanz = [Dmc_list[i].pos_from(Dmc_list[j]).magnitude() for i, j in itt.permutations(range(n), r=2)]
Dmc_distanz_lam = sm.lambdify(x_list + y_list, Dmc_distanz, cse=True)

force1_lam = []
for i in range(n):
      impact = impact_force_street(i, epsilons_list[i], rhodts_list)
      force1_lam.append(sm.lambdify(qL + pL, [ me.dot(impact, uv) for uv in (N.x, N.y)], cse=True))
print(f'it took {time.time() - start1 :.5f} sec to do the lambdification')

**Numerical integration**
- the parameters and the initial values of independent coordinates are set. The ellipses are placed at random, and then I check that they do not overlap with each other. For this, I assume for simplicity that the ellipses are circles with radius $R_0 = a \bigvee b$
- an exception is raised if $\alpha$ or $\beta$ are selected such that the particle will be outside of the ellipse. 
- I check whether the minimum osculating cycle of the street is smaller than the max. osculating circle of the ellipse. If it is smaller, second contact points may occur.

I plot the initial location of the ellipse. This plot also gives possible contact points. The closest one is marked on the street.\
Also, the relative initial generalized speeds are indicated.

In [ ]:
start2 = time.time()

#=============================================
# Input parameters
#=============================================
m1  = 1.
mo1 = 1.
g1  = 9.8
a1  = 2.
b1  = 1.

amplitude1  = 1.
frequenz1   = 0.25

EYe1        = 1.e7  
EYs1        = 1.e7
ctau1       = 0.9
nue1        = 0.28
nus1        = 0.28

alpha_list1 = [0.5 for _ in range(n)]
beta_list1  = [0.5 for _ in range(n)]

intervall = 0. 1                            
X1        = 4.                                        # Initial locations must be between -X1, X1 in X - coordinates
Y1        = 11.                                       # initial Y - coordinates must be between the street and Y1

min_winkel = 0.1                                      # max sin(angle) how the tangents of the street and the ellipse may differ for a contact point
np.random.seed(123456789)                             # for any other random configurtation I tried, it wourd rum 'forever'! (like I aborted after it had not finished 2 sec of simulation in 10 hours of running)


#-----------------------------------------------------------------------------------------------------------------------------------------------------------------
# ensure the observers are inside the ellipses
lokation = np.array([(alpha_list1[i]/a1)**2 + (beta_list1[i]/b1)**2 - 1. for i in range(n)])
if any(i > 0. for i in lokation):
    raise Exception('Particle is outside the ellipse')

#-------------------------------------------------------------------------------------------------------------------------------------------------------------------
q_list1 = [*np.random.choice(np.linspace(-np.pi, np.pi, 100), size=n)]                  # initial angle of the ellipses
alpha_list1    = [0.5 for _ in range(len(alpha_list))]                                  # location of observer
beta_list1     = [0.5 for _ in range(len(beta_list))]                                   # dto.
epsilons_list1 = [0. for _ in range(n)]                                                 # values of no consequence
xs_list1       = [1. for _ in range(n)]                                                 # values of no consequence
ls_list1       = [1. for _ in range(n)]                                                 # values of no consequence

rhodte_list1     = [1. for _ in range(n*(n-1))]                                         # values of no consequence
richtung_list1   = [1. for _ in range(n*(n-1))]                                         # values of no consequence


ux_list1 = list((-1.5)**(i+1) for i in range(n))                                        # initial speed of center of the ellipse in x direction
uy_list1 = [(-1.5)*(i+1) for i in range(n)]                                             # initial speed of center of the ellipse in Z direction
u_list1  = list(-1. * np.random.choice(np.linspace(-5., 5., 100), size=n))              # initial rotationmal speed
 
schritte = int(intervall * 40000)

#-------------------------------------------------------------------------------------------------------------------------------------------------------------------
# 1. randomly place the ellipses as described above
# This is needed to check, whether an ellipse collides with the street.
# it will only 'work' if the ellipse intersects the street. If the ellispe is placed on the wrong side of the street entirely, it will not notice this mistake.
# I did not bother to catch this error.
P1 = me.Point('P1')
x  = sm.symbols('x')
P1.set_pos(O, x*N.x + gesamt1(x, amplitude, frequenz)*N.y)
kollision_lam = []
for i in range(n):
    kollision = Dmc_list[i].pos_from(P1).magnitude()
    kollision_lam.append(sm.lambdify([x_list[i], y_list[i]] + [x, amplitude, frequenz], kollision, cse=True))

zaehler = 0
r01     = max(a1, b1)
while zaehler <= 500:
    zaehler += 1
    try:
        x_listen = []
        y_listen = []
        test1    = True
        for i in range(n):
            x_listen.append(np.random.choice(np.linspace(-X1 + r01, X1 - r01, 100)))
            hilfs = np.linspace(x_listen[-1] - r01, x_listen[-1] + r01, 100)
            hilfs_max = np.max(gesamt_lam(hilfs, amplitude1, frequenz1))
            y_listen.append(np.random.choice(np.linspace(gesamt_lam(x_listen[-1], amplitude1, frequenz1) + 1.*r01, Y1 - r01, 100)))
            test1 = test1 and np.all(np.array(kollision_lam[i](hilfs, y_listen[i], hilfs, amplitude1, frequenz1 )) > 1.*r01 )

        test = np.all(np.array(Dmc_distanz_lam(*x_listen, *y_listen)) - 3.*r01 > 0.)
        x_list1 = x_listen
        y_list1 = y_listen
        
        if test == True and test1 == True:
            raise Rausspringen
    except:
        break

if zaehler <= 500:
    print(f'it took {zaehler} rounds to get valid initial conditions')
else:
    raise Exception(' no good location for ellipses found, make X1 and/or Y1 larger, or try again.')
#--------------------------------------------------------------------------------------------------------------------------------------------------------------------
#find the largest admissible r_max, given strasse, amplitude, frequenz
r_max = max(a1**2/b1, b1**2/a1)  # max osculating circle of an ellipse
def func2(x, args):
# just needed to get the arguments matching for minimize
    return np.abs(r_max_lam(x, *args))

x0 = 0.1            # initial guess
minimal = minimize(func2, x0, [amplitude1, frequenz1])
if r_max < (x111 := minimal.get('fun')):
    print('selected r_max of the ellipse = {} is less than the minimal osculating circle of the street = {:.2f}'
          .format(r_max, x111), '\n')
else:
    print('selected r_max of the ellipse =  {} is larger than the minimal osculating circle of the street = {:.2f}'
          .format(r_max, x111), '\n')

#---------------------------------------------------------------------------------------------------------------------------------------------------------------------- 
# numerically find x1 = X coordinate of CPhs and l1 := distance from CPh to CPhs for the initial condition
# and make a plot of the initial situation
def func_x1_l1(x0, args):
    ergebnis = equation_street_lam[kk](*x0, *args)
    return ergebnis.reshape(2)

def funce(x0, args):
    return min_distanzCPheiCPhej_lam[zaehler](*x0, *args)

def func_jakob(x0, args):
    return jakob_lam[zaehler](*x0, *args).reshape(2)



# This is to asign colors of 'plasma' to the ellipses.
Test = mp.colors.Normalize(0, n)
Farbe = mp.cm.ScalarMappable(Test, cmap='plasma')
farben = [Farbe.to_rgba(l) for l in range(n)]    # color of the starting position

fig, ax = plt.subplots(figsize=(10, 10))
ax.set_aspect('equal')
ax.set_title(f'possible contact points are where the tangents of street and ellipse differ by less than {np.rad2deg(np.arcsin(min_winkel)):.1f}° \n The dots on the street indicates the closest one \n The magenta arrows indicate the (relative) initial speeds \n' +
             f'The white dots are the particles' )
 
for kk in range(n):
    TEST = []
    TEST1 = []
    x0 = list((-100., 100.))
    for epsilon in np.linspace(1.e-15, 2.*np.pi, int(25/min_winkel)):
        epsilons_list1[kk] = epsilon
        if nhat_lam[kk](*q_list1, *epsilons_list1, a1, b1)[1] <= 0.:
            args1 = q_list1 + x_list1 + y_list1 + [xs_list1[j] for j in range(n) if j != kk] + [ls_list1[j] for j in range(n) if j != kk] + epsilons_list1 + [a1, b1, amplitude1, frequenz1]
            ergebnis = root(func_x1_l1, x0, args1, method='broyden1')
            x0 = ergebnis.x
            xs_list1[kk] = x0[0]
            ls_list1[kk] = x0[1]
   
            if parallel_street_lam[kk](*q_list1, *xs_list1, *epsilons_list1, *ls_list1, a1, b1, amplitude1, frequenz1) < min_winkel:
                TEST1.append(liste_lam[kk](*q_list1, *x_list1, *y_list1, *epsilons_list1, a1, b1))
                TEST.append((*x0, epsilon))
    kontakt = min(TEST, key = lambda k: k[1])
    epsilons_list1[kk] = kontakt[2]
    xs_list1[kk]       = kontakt[0]
    ls_list1[kk]       = kontakt[1]
     

    Cax = np.array([TEST1[i][0][0] for i in range(len(TEST1))])
    Cay = np.array([TEST1[i][0][1] for i in range(len(TEST1))])
    Cex = np.array([TEST1[i][1][0] for i in range(len(TEST1))])
    Cey = np.array([TEST1[i][1][1] for i in range(len(TEST1))])

    elli = patches.Ellipse((x_list1[kk], y_list1[kk]), width=2.*a1, height=2.*b1, angle=np.rad2deg(q_list1[kk]), zorder=1, fill=True, color=farben[kk], ec='black')
    ax.add_patch(elli)
    weite = X1 + max(a1, b1)
    ax.plot(x_list1[kk], y_list1[kk], color='yellow', marker='o', markersize=2)
    ax.plot(Po_ort_lam[kk](*q_list1, *x_list1, *y_list1, a1, b1, *alpha_list1, *beta_list1)[0], Po_ort_lam[kk](*q_list1, *x_list1, *y_list1, a1, b1, *alpha_list1, *beta_list1)[1], color='white', marker='o', markersize=5)
    ax.plot(kontakt[0], gesamt1(kontakt[0], amplitude1, frequenz1), color=farben[kk], marker='o', markersize=7)
    ax.plot(np.linspace(-weite, weite, 100), gesamt_lam(np.linspace(-weite, weite, 100), amplitude1, frequenz1), color='blue')
    for i in range(len(Cax)):
        x_werte = [Cax[i], Cex[i]]
        y_werte = [Cay[i], Cey[i]]
        ax.plot(x_werte, y_werte)
        ax.arrow(Cax[i], Cay[i], Cex[i]-Cax[i], Cey[i]- Cay[i], shape='full', width=0.025)

# plot the initial speeds
ux_np = np.array(ux_list1)
uy_np = np.array(uy_list1)
u_np  = np.array(u_list1)
# normalize the initial speed arrows.
groesse = max(np.max(np.abs(ux_np)), np.max(np.abs(uy_np)))
if np.abs(groesse) < 1.e-5:
    groesse = 1.
ux_np = ux_np/groesse * 2. * max(a1, b1)    # If I do not multiply with max(a1, b1) the arrows are too short to be seen properly
uy_np = uy_np/groesse * 2. * max(a1, b1)
u_np  = u_np / np.max(np.abs(u_np))

# plot the linear speed arrows
style = "Simple, tail_width=0.5, head_width=4, head_length=8"
kw = dict(arrowstyle=style, color="magenta")
for i in range(n):
    a11 = patches.FancyArrowPatch((x_list1[i], y_list1[i]), (x_list1[i] + ux_np[i], y_list1[i] + uy_np[i]), **kw)
    ax.add_patch(a11)

# plot the curved arrows for the rotational speed
for j, i in enumerate(u_np):
    farbe = 'magenta'
    if i >= 0.: 
        zeit = np.linspace(0., i * 1.9 * np.pi, 100)
        xk = 1.5 * b1 * np.cos(zeit) + x_list1[j]
        yk = 1.5 * b1 * np.sin(zeit) + y_list1[j]
        ax.plot(xk, yk, color = farbe)
        ax.arrow(xk[95], yk[95], xk[95] - xk[94], yk[95] - yk[94], shape='full', lw=0.5, length_includes_head=True, head_width=0.5, color=farbe)
    else:
        zeit = np.linspace(2.*np.pi, (1+i) * 1.9 * np.pi, 100)
        xk = 1.5 * b1 * np.cos(zeit) + x_list1[j]
        yk = 1.5 * b1 * np.sin(zeit) + y_list1[j]
        ax.plot(xk, yk, color = farbe)
        ax.arrow(xk[95], yk[95], xk[95] - xk[94], yk[95] - yk[94], shape='full', lw=0.5, length_includes_head=True, head_width=0.5, color=farbe)

# find possible collision points between the ellipses
zaehler        = -1
TEST3          = []
le_hilfs       = []
epsilone_hilfs = []
le_list1       = []
epsilone_list1 = []

for i, j in itt.permutations(range(n), r=2):
    zaehler += 1
    x0 = (0., 0.)                                                       # initial guess
    args1 = q_list1 + x_list1 + y_list1 + [a1, b1]
# here I use the minimize() method, as I do not have a good guess for epsilon_i, epsilon_j to minimize the distance
# in the integration, I use the gradient method, as there are good initial guesses available, and the gradient method seems faster and more accurate.
    epsilon_min = minimize(funce, x0, args1, jac=func_jakob, tol=1.e-12)
    min_eps = epsilon_min.x
    ll_min = funce(min_eps, args1)
    TEST3.append([ll_min, i, j, min_eps[0], min_eps[1], zaehler])
    le_hilfs.append(ll_min)
    le_list1.append(ll_min)
    epsilone_hilfs.append((min_eps[0], min_eps[1]))                     # needed later to re - calculate the 'contact epsilon' and the distances from ellipse_i to ellipse_j 
    epsilone_list1.append((min_eps[0], min_eps[1]))

for kk in range(len(TEST3)):
    koerper1 = TEST3[kk][1]
    koerper2 = TEST3[kk][2]
    epsilon1 = TEST3[kk][3]
    epsilon2 = TEST3[kk][4]
    x11 = CPhe_list_lam(*q_list1, *x_list1, *y_list1, a1, b1, epsilon1)[koerper1][0]
    x12 = CPhe_list_lam(*q_list1, *x_list1, *y_list1, a1, b1, epsilon2)[koerper2][0]
    y11 = CPhe_list_lam(*q_list1, *x_list1, *y_list1, a1, b1, epsilon1)[koerper1][1]
    y12 = CPhe_list_lam(*q_list1, *x_list1, *y_list1, a1, b1, epsilon2)[koerper2][1]
    
    ax.plot([x11, x12], [y11, y12], color=farben[koerper1], linestyle='dotted')

#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
# fill in balance parameters
print('angles on ellipse for contact with street: {}°'.format([np.rad2deg(epsilons_list1[i]) for i in range(n)]))
rhodts_list1 = []
for i in range(n):
    rhodts_list1. append(rhodts_lam[i](*q_list1, *x_list1, *y_list1, *u_list1, *ux_list1, *uy_list1, *epsilons_list1, a1, b1, amplitude1, frequenz1))
print('impact speeds:                            ', rhodts_list1)
print('potential X coordinates of contact point: ', xs_list1)
print('\n')
for i in range(n):
    print(f'initial distance of ellise_{i} from the street is {ls_list1[i]:.3f}')
print('\n')
zaehler = -1
for i, j in itt.permutations(range(n), r=2):
    zaehler += 1
    if i < j:
        print(f'Measured in their body fixed frames, {np.rad2deg(epsilone_list1[zaehler][0]):.1f}° and {np.rad2deg(epsilone_list1[zaehler][1]):.1f}° are the angles at which ellipse_{i} and ellipse_{j} would collide')

zaehler = -1
for i, j in itt.permutations(range(n), r=2):
    zaehler += 1
    if i < j:
        print(f'the initial distance of ellipse_{i} and ellipse={j} is {le_list1[zaehler]:.3f}')
print('\n')

zaehler = -1
for i, j in itt.permutations(range(n), r=2):
    zaehler += 1
    args1 = [epsilone_list1[zaehler][0], epsilone_list1[zaehler][1]] + q_list1 + x_list1 + y_list1 + [a1, b1]
    richtung_list1[zaehler] = richtung_lam[zaehler](*args1)

# pL    = [m, mo, g, a, b, amplitude, frequenz] + [ctau, EYe, EYs, nue, nus] + xs_list + epsilons_list + ls_list + alpha_list + beta_list + rhodts_list + epsilone_list + le_list + rhodte_list + richtung_list
pL_vals = [m1, mo1, g1, a1, b1, amplitude1, frequenz1] + [ctau1, EYe1, EYs1, nue1, nus1] + xs_list1 + epsilons_list1 + ls_list1 + alpha_list1 + beta_list1 + rhodts_list1 + epsilone_list1 + le_list1 + rhodte_list1 + richtung_list1

y0 = q_list1 + x_list1 + y_list1 + u_list1 + ux_list1 + uy_list1
print('\n')
print('starting values are:   ', y0)
print('\n')
print(f'it took {(time.time() - start2):.3f} sec to get valid initial conditions')


Here the actual **integration** starts\
1.
I look for the contact point of the ellipses with the street as described above.\
I start considering only int(zaehler / min winkel) points around the ellipse. If this does not find a 'good' point, I try with more points.\
This is done with the *try...raise Exception()* loop.\
2.
I try to minimize the distance between ellipses by solving $\dfrac{d}{d\epsilon_i} |{}^{CPhe_i} \bar r^{CPhe_j}(\epsilon_i, \epsilon_j)|$ and $\dfrac{d}{d\epsilon_j} |{}^{CPhe_i} \bar r^{CPhe_j}(\epsilon_i, \epsilon_j)|$, for $\epsilon_i, \epsilon_j$\
As this is oinly a necessary condition, the correct solution depends on the initial guess. Very near to a collision, I switch from the $\nabla$ method to using minimize(..), with the Jacobian, and a small tolerance.\
This seems better near a contact point.\
3.
Unless I make max_step very small, the integration does not work.\
I assume, of course I do not know!, that the reason is this: finding the distances is not very accurate and may not depend continuously on the coordinates. So, when *solve_ipv* tries to improve the accuracy by switching back and forth in time, and changing the step size, this inaccuracy makes it fail.\
(of course only important when the penetration depths are negative - but then again, without collisions, this would be a very simple simulation)


In [ ]:
#==============================
cut_off = 90000
zaehler    =  5
max_step = 0.0001 
#==============================
start1 = time.time()
         
zaehler1      = zaehler
nixwars       = [0] * n
kontaktes     = [[] for _ in range(n)]
fehler        = 0
call_minimize = 0

zeitee    = [[] for _ in range(n*(n-1))]
laengeee  = [[] for _ in range(n*(n-1))]     
epsilonee = [[] for _ in range(n*(n-1))]     

def func_x1_l1_1(x0, args):
    kk = args[-1]
    args1 = [args[jj] for jj in range(len(args) - 1)]
    ergebnis = equation_street_lam[kk](*x0, *args1)
    return ergebnis.reshape(2)

def func_x1_l1_1_jakob(x0, args):
    kk = args[-1]
    args1 = [args[jj] for jj in range(len(args) - 1)]
    ergebnis = equation_street_jakob_lam[kk](*x0, *args1)
    return ergebnis


def funce1didj(x0, args):
    zaehler = args[-1]
    args1 = [args[i] for i in range(len(args)-1)]
    return min_distanzCPheiCPhej_lam1[zaehler](*x0, *args1)

def funce1didj_jakob(x0, args):
    zaehler = args[-1]
    args1 = [args[i] for i in range(len(args)-1)]
    return jakob_lam1[zaehler](*x0, *args1)


def funce1(x0, args):
    zaehler = args[-1]
    args1 = [args[i] for i in range(len(args)-1)]
    return min_distanzCPheiCPhej_lam[zaehler](*x0, *args1)

def funce1_abs(x0, args): 
# the 'unsigned distance between ellipse. Used when the ellipse are very close to each other
    zaehler = args[-1]
    args1 = [args[i] for i in range(len(args)-1)]
    return np.abs(min_distanzCPheiCPhej_lam[zaehler](*x0, *args1))

def func_jakob1(x0, args):
    zaehler = args[-1]
    args1 = [args[i] for i in range(len(args)-1)]
    return jakob_lam[zaehler](*x0, *args1).reshape(2)
   
#@profile
def ellipse_street(y, args, kk, zaehler1):
    TEST = []   #TEST = List.empty_list(types.float64)
    x0  = (args[12 + kk], args[12 + 2*n + kk])                    
    for epsilon in np.linspace(1.e-15, 2.*np.pi, int(zaehler1/min_winkel)):
        args[12 + n + kk] = epsilon
        if nhat_lam[kk](*[y[i] for i in range(n)], *[args[12 + n + jj] for jj in range(n)], a1, b1)[1] <= 0.:
            args1 = [y[i] for i in range(3*n)] + [args[12 + jj] for jj in range(n) if jj != kk] + [args[12 + 2*n +jj] for jj in range(n) if jj != kk] + [args[12 + n + jj] for jj in range(n)] + [a1, b1, amplitude1, frequenz1] + [kk]
            ergebnis = root(func_x1_l1_1, x0, args1, jac=func_x1_l1_1_jakob)
            args[12 + kk]       = ergebnis.x[0]
            args[12 + 2*n + kk] = ergebnis.x[1]
                
            if parallel_street_lam[kk](*[y[jj] for jj in range(n)], *[args[12 + jj] for jj in range(3*n)], a1, b1, amplitude1, frequenz1) < min_winkel:
                TEST.append((*ergebnis.x, epsilon))
    
    return TEST



#@profile
def gradient(t, y, args):
    global zaehler1, nixwars, fehler, call_minimize

# find possible collision points between the ellipses, and also find rhodtemax
# pL_vals = [m1, mo1, g1, a1, b1, amplitude1, frequenz1] + [ctau1, EYe1, EYs1, nue1, nus1] + xs_list1 + epsilons_list1 + ls_list1 + alpha_list1 + beta_list1 + rhodts_list1 + epsilone_list1 + le_list1 + rhodte_list1 + richtung_list1
    zaehler3 = -1  
    for _ in range(n*(n-1)):
        zaehler3 += 1
        x0 = args[12 + 6*n + zaehler3]
        if args[12 + 6*n + n*(n-1) + zaehler3] <= 0.0002      :
            args1 = [y[ij] for ij in range(3*n)] + [a1, b1] + [zaehler3]
# if the ellipses are very close to each other, I switch to minimize: 
# - as this does not happen often, it does not slow down the simulation too much
# - it better finds the distance.
# Note, that I niminize the 'unsigned' distance
            epsilon_min = minimize(funce1_abs, x0, args1, jac=func_jakob1, tol=1.e-10)
            call_minimize += 1
            if epsilon_min.success != True:
                fehler += 1
#                print(f'mimimize message is: {epsilon_min.message}, it happened at time t = {t:.3f}, integration time {(time.time() - start1):.3f}')
            min_eps = epsilon_min.x % (2. * np.pi)
            args[12 + 6*n + zaehler3]           = (min_eps[0], min_eps[1])

        else:
            args1 = [y[ij] for ij in range(3*n)] + [a1, b1] + [zaehler3]
            epsilon_min = root(funce1didj, x0, args1, jac=funce1didj_jakob)
            min_eps = epsilon_min.x % (2.* np.pi)
             
        ll_min = funce1(min_eps, args1)
        args[12 + 6*n + n*(n-1) + zaehler3] = ll_min
        args[12 + 6*n + zaehler3]           = (min_eps[0], min_eps[1])
        laengeee[zaehler3].append(ll_min)
        epsilonee[zaehler3].append([min_eps[0], min_eps[1]])
        zeitee[zaehler3].append(t)

        if 0. <= ll_min <= 0.01: 
            args1 = [args[12 + 6*n + jj] for jj in range(n*(n-1))] + [args[12 + 6*n + 3*n*(n-1) + jj] for jj in range(n*(n-1)) ] + [a1, b1]
            rhodteiej = rhodte_lam[zaehler3](*y, *args1)
            args[12 + 6*n + 2*n*(n-1) + zaehler3] = rhodteiej
        
# If the distance between the contact points becomes very small, calculating the direction CPhe_i.pos_from(CPhej) seems to become numerically difficult. This allows me to fix the direction once 
# the distance is small. Of course, mechanically speaking not correct.
        if ll_min > 2.e-4:
            args1 = [args[12 + 6*n + zaehler3][0], args[12 + 6*n + zaehler3][1]]  + [y[jj] for jj in range(3*n)] + [a1, b1]
            args[12 + 6*n + 3*n*(n-1) + zaehler3] = richtung_lam[zaehler3](*args1)

# After penetration, the direction has to be reversed to ensure that the contact is the force of ellipse_i on ellipse_j 
        elif ll_min < -2.e-4:
             args1 = [args[12 + 6*n + zaehler3][0], args[12 + 6*n + zaehler3][1]]  + [y[jj] for jj in range(3*n)] + [a1, b1]
             args[12 + 6*n + 3*n*(n-1) + zaehler3] = (-richtung_lam[zaehler3](*args1)[0], -richtung_lam[zaehler3](*args1)[1])
        else:
            pass
#==========================================================================================================================================================================================================
    
# numerically determine the closest potential contact point of the ellipses to the street
# pL_vals = [m1, mo1, g1, a1, b1, amplitude1, frequenz1] + [ctau1, EYe1, EYs1, nue1, nus1] + xs_list1 + epsilons_list1 + ls_list1 + alpha_list1 + beta_list1 + rhodts_list1 + epsilone_list1 + le_list1 + rhodte_list1 + richtung_list1
       
    for kk in range(n):
 
        while nixwars[kk] < cut_off:
            try:
                TEST = ellipse_street(y, args, kk, zaehler1)
                
                if len(TEST) > 0:
                    raise Rausspringen
                zaehler1 = 5 * zaehler1
                nixwars[kk] += 1
        
                if nixwars[kk] > cut_off: 
                    raise Exception(f'At {t:.3f} sec fsolve(..) did not find a solution for the {nixwars[kk]}th time for ellipse_{kk}. Hence integration was terminated')
#                print(f'at time {t:.6f} no contact point was found immediately. Totally {nixwars} such occurences so far, running time is {(time.time() - start1):.3f}, zaehler1 is {zaehler1}')


            except:            
                kontakt = min(TEST, key = lambda k: k[1])
                args[12 + n + kk]   = kontakt[2]                    #epsilons_list1[kk] = kontakt[2]
                args[12 + kk]       = kontakt[0]                    #xs_list1[kk]       = kontakt[0]
                args[12 + 2*n + kk] = kontakt[1]                    #ls_list1[kk]       = kontakt[1]
                zaehler1 = zaehler
                kontaktes[kk].append(kontakt)
                break

# determine the speed right at the impact time           
        if 0. <= args[12 + 2*n + kk] <= 0.1:
            args[12 + 5*n + kk] = rhodts_lam[kk](*y, *[args[12 + n + jj] for jj in range(n)], a1, b1, amplitude1, frequenz1) 
                    
    sol = np.linalg.solve(MM_lam(*y, *args), force_lam(*y, *args))
    return np.array(sol).T[0]


times = np.linspace(0., intervall, schritte)
t_span = (0., intervall)

resultat1 = solve_ivp(gradient, t_span, y0, t_eval = times, args=(pL_vals,), max_step=max_step) #, method='BDF' , atol=1.e-4, rtol=1.e-4) 
resultat = resultat1.y.T
print('Shape of result: ', resultat.shape)
event_dict = {-1: 'Integration failed', 0: 'Integration finished successfully', 1: 'some termination event'}
print(event_dict[resultat1.status], ' the message is: ', resultat1.message)
print(f'on {nixwars} occasions a contact point with the street was not found immediately' )
if call_minimize != 0:
    print(f'Trying to find the distance between ellipses minimize(...) had some problem in {fehler} cases, in {(fehler/call_minimize * 100):.3f} % of total calls of minimize')
print('\n')
print('the integration made {} function calls. It took {:.3f} sec'.format(resultat1.nfev, time.time() - start1))
profiler.print_stats() 

Initially, I had problems finding the distances of the ellipses from each other, so I made this plot to help me find out where the problem might be.\
Now it mostly works, but I felt no need to delete it.

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
bishin = len(zeitee[0])
zaehler = -1
for i, j in itt.permutations(range(n), r=2):
    zaehler += 1
#    if j > i:
    ax.plot(zeitee[zaehler][: bishin], laengeee[zaehler][:bishin], label=f'distance of ellipse_{i} from ellipse_{j}')
ax.set_title('Distances of ellipses from each other, as calculated during integration')
ax.set_xlabel('time (sec)')
ax.set_ylabel('distance (m)')
ax.legend();

Plot the generalized coordinates you want to see.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
bezeichnung = [str(i) for i in qL]
zeigen = [j for j in range(2*n, 3*n)] + [j for j in range(5*n, 6*n)]
for i in zeigen:
    ax.plot(times[: resultat.shape[0]], resultat[:, i], label=bezeichnung[i])
ax.set_xlabel('time (sec)')
ax.set_ylabel('units depend on which gen. coordinates were selected')
ax.set_title('generalized coordinates')
ax.legend();

**Location and distance of contact point**\
The distance is needed for the spring energy.\
The closest distance to a possible contact point is available only during numerical integration.\
I collect them during integration, and match them as closely as possible to the points in time given by the result.
- *epsilonee* holds the contact angles of $ellipse_i$ and $ellipse_j$ during each step of the numerical integration
- *laengeee* holds the distance between $ellipse_i$ and $ellipse_j$ during each step of the numerical integration
- *kontaktes* holds (among other data) the distance of $ellipse_i$ from the street.
- *zeitee* holds the time of the storage in the lists above
- *indexe* holds the location in *zeitee* closest to the respective time in times. 

All else is pretty obvious.

**NOTE**: I only look at $\dfrac{zaehler}{min_{winkel}}$ ( = 50 with my parameters) positions on the circumference of the ellipse to find the closest distance to the street. Hence, it may not find the distance, but some other point on the street, where the 'tangent criterium' is met.\
This only seems to happen when the elllipse is far away from the street, so it does not matter. If it is close to the street, it seems to find the closest distance. The distance is only needed for the EOM if it is negative. 

In [ ]:
zeitee = np.array(zeitee)

indexe  = []

for zeit in times:
    zaehler1 = np.min(np.argwhere(np.array(zeitee[0, :] >= zeit)))
    indexe.append(zaehler1)

if len(indexe) != len(times):
    raise Exception('Something happened')

laenge = [[] for _ in range(n*(n-1))]
winkel = [[] for _ in range(n*(n-1))]

zaehler = -1
for _ in range(n*(n-1)):
    zaehler += 1
    for i in range(len(times)):
        winkel[zaehler].append(epsilonee[zaehler][indexe[i]])
        laenge[zaehler].append(laengeee[zaehler][indexe[i]])
kontaktee = np.array(laenge)
winkel    = np.array(winkel)

kontakte = [[] for _ in range(n)]
for kk in range(n):
    for i in range(len(times)):
        kontakte[kk].append(kontaktes[kk][indexe[i]])
kontakte = np.array(kontakte)
    
fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, figsize=(10,20))
for kk in range(n):
    ax1.plot(times, kontakte[kk, :, 1], label ='ellipse_' + str(kk))
ax1.set_title('distance of the ellipses from the street')
#ax.set_xlabel('time (sec)')
ax1.set_ylabel('distance (m)')
ax1.legend()

#fig, ax = plt.subplots(figsize=(10,5))
test1 = [[] for _ in range(n)]
test2 = [[] for _ in range(n)]
zaehler = [0. for _ in range(n+1)]
for kk in range(n):
    for i in  range(kontakte.shape[1]):
        if kontakte[kk, i, 1] <= 0.:
            test1[kk]. append(times[i])
            test2[kk].append(kontakte[kk, i, 1])
            zaehler[kk] += 1
        else:
            test1[kk]. append(times[i])
            test2[kk].append(0.)


    ax2.plot(test1[kk], test2[kk], label='ellipse_' + str(kk))
ax2.set_title('Penetration close up view')
#ax2.set_xlabel('time (sec)')
ax2.set_ylabel('penetration depth (m)')
ax2.legend()
for kk in range(n):
    print(f'There are {zaehler[kk]} points where penetration between ellipse_{kk} and street takes place, {zaehler[kk]/schritte * 100:.3f} % of total points')
#---------------------------------------------------------------------------------------------------------------------------------------
#fig, ax = plt.subplots(figsize=(10,5))
zaehler3 = -1
for i, j in itt.permutations(range(n), r=2):
    zaehler3 += 1
#    if i < j:
    ax3.plot(times, kontaktee[zaehler3, :], label =f'ellipse_{i} from ellipse_{j}')
ax3.set_title('distance of contact point from the ellipses')
#ax3.set_xlabel('time (sec)')
ax3.set_ylabel('distance (m)')
ax3.legend()

#fig, ax = plt.subplots(figsize=(10,5))
test1 = [[] for _ in range(n*(n-1))]
test2 = [[] for _ in range(n*(n-1))]
zaehler = [0. for _ in range(n*(n-1)+1)]
for kk in range(n*(n-1)):
    for i in  range(kontaktee.shape[1]):
        if kontaktee[kk, i] <= 0.:
            test1[kk]. append(times[i])
            test2[kk].append(kontaktee[kk, i])
            zaehler[kk] += 1
        else:
            test1[kk]. append(times[i])
            test2[kk].append(0.)

zaehler3 = -1
for i, j in itt.permutations(range(n), r=2):
    zaehler3 += 1
#    if i < j:
    ax4.plot(test1[zaehler3], test2[zaehler3], label=f'ellipse_{i} from ellipse_{j}')
ax4.set_title('Penetration close up view')
ax4.set_xlabel('time (sec)')
ax4.set_ylabel('penetration depth (m)')
ax4.legend()
zaehler3 = -1
for i, j in itt.permutations(range(n), r=2):
    zaehler3 += 1
    if i < j:
        print(f'There are {zaehler[zaehler3]} points where penetration between ellipse{i} and ellipse_{j} takes place, {zaehler[zaehler3]/schritte * 100:.3f} % of total points')

**Energies of the system**\
For $c_{\tau} = 1$ total energy should be constant, else it should drop monotonically.\
It does not always do so, numerical problems? I do not know.

In [ ]:
kin_np    = np.empty(schritte)    
pot_np    = np.empty(schritte)
spring_np = np.empty(schritte)
total_np  = np.empty(schritte)

pL_vals2 = copy.deepcopy(pL_vals)

for i in range(schritte):
    for kk in range(n):
        pL_vals2[12 + kk]        = kontakte[kk, i, 0]
        pL_vals2[12 + 2*n + kk]  = kontakte[kk, i, 1]
        pL_vals2[12 + n + kk]    = kontakte[kk, i, 2]

    for kk in range(n*(n-1)):
        pL_vals2[12 + 6*n + n*(n-1) + kk] = kontaktee[kk, i]
        pL_vals2[12 + 6*n + kk]           = winkel[kk, i]

    kin_np[i]      =    kin_lam(*[resultat[i, j] for j in range(resultat.shape[1])], *pL_vals2)
    pot_np[i]      =    pot_lam(*[resultat[i, j] for j in range(resultat.shape[1])], *pL_vals2)
    spring_np[i]   = spring_lam(*[resultat[i, j] for j in range(resultat.shape[1])], *pL_vals2)
    total_np[i]    = kin_np[i] + pot_np[i] + spring_np[i]
        
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(times, pot_np, label='potential energy')
ax.plot(times, kin_np, label='kinetic energy')
ax.plot(times, spring_np, label='spring energy')
ax.plot(times, total_np, label='total energy')
ax.set_xlabel('time (sec)')
ax.set_ylabel("energy (Nm)")
ax.set_title(f'Energies of the system, with ctau = {ctau1}')
ax.legend();
total_max = np.max(total_np)
total_min = np.min(total_np)
if ctau1 == 1.:
    print('max deviation of total energy from being constant is {:.2e} % of max total energy'.
          format((total_max - total_min)/total_max * 100))

**Animate** the motion of the ellipse\
The dotted lines show the 'closest' contact point, that is the point where the ellipse would touch the street if it were 'blown up' to just touch the street at this specific point in time.\
(If the ellipse is far away from the street, the above statement may not be true, see explanation above.)\
The lines connecting the ellipses...as above.\
As HCML is slow, I limit to number of points considered to around *schrittzahl*

In [ ]:
#======================
schrittzahl = 2500
#======================
# reduce the number of points considered to around schrittzahl
faktor = max(1, int(resultat.shape[0] / schrittzahl))
resultat1 = []
times1    = []
kontakte1 = [[] for _ in range(n)]
for i in range(resultat.shape[0]):
    if i % faktor == 0:
        resultat1.append(resultat[i, :])
        times1.append(times[i])
        
for kk in range(n):
    for i in range(resultat.shape[0]):
        if i % faktor == 0:
            kontakte1[kk].append(kontakte[kk, i, 0])

schritte1 = len(times1)
resultat1 = np.array(resultat1)
print(f'Here, {schritte1} ponts in time are considerted')
# contact points / lines of ellipse_i with ellipse_j
# 1.) reduce the points in time considered to around schrittzahl
zaehler = -1
winkel_wenig = [[] for _ in range(n*(n-1))]
for jj, jjj in itt.permutations(range(n), r=2):
    zaehler += 1
    for i in range(resultat.shape[0]):
        if i % faktor == 0:
            winkel_wenig[zaehler].append(winkel[zaehler, i])

winkel_np = np.array(winkel_wenig)

# 2.) Get the coordinates of the potential impact points of ellipse_i with ellipse_j, sort them, so their connecting line can be plotted
zaehler = -1
delta_list= list(list(sm.symbols(f'delta{i}{j}, delta{j}{i}')) for i, j in itt.permutations(range(n), r=2))
coords_list = []
coords_lam = []
for i, j in itt.permutations(range(n), r=2):
    zaehler += 1
    CPi, CPj = sm.symbols('CPi, CPj', cls=me.Point)
    CPi.set_pos(Dmc_list[i], a * sm.cos(delta_list[zaehler][0])*A_list[i].x + b * sm.sin(delta_list[zaehler][0])*A_list[i].y)
    CPj.set_pos(Dmc_list[j], a * sm.cos(delta_list[zaehler][1])*A_list[j].x + b * sm.sin(delta_list[zaehler][1])*A_list[j].y)
    coords = [[me.dot(CPi.pos_from(O), uv) for uv in (N.x, N.y)], [me.dot(CPj.pos_from(O), uv) for uv in (N.x, N.y)]]
    coords_lam.append(sm.lambdify(q_list + x_list + y_list + delta_list[zaehler] +  [a, b], coords, cse=True)) 

coords_list  = []
for i in range(n*(n-1)):
    coords_list1 = []
    for j in range(schritte1):
        coords_list1.append(coords_lam[i](*[resultat1[j, jj] for jj in range(3*n)], *winkel_np[i, j], a1, b1))
    coords_list.append(coords_list1)
coords_list = np.array(coords_list)

X_werte_elli = [[] for _ in range(n*(n-1))]
Y_werte_elli = [[] for _ in range(n*(n-1))]

for zaehler in range(n*(n-1)):
    for i in range(schritte1):
        X_werte_elli[zaehler].append([coords_list[zaehler, i, 0, 0], coords_list[zaehler, i, 1, 0]])
        Y_werte_elli[zaehler].append([coords_list[zaehler, i, 0, 1], coords_list[zaehler, i, 1, 1]])

X_werte_elli = np.array(X_werte_elli)
Y_werte_elli = np.array(Y_werte_elli)

# 3.) get the coordinates of the points, for better visibility
X_point = np.empty((X_werte_elli.shape[0], X_werte_elli.shape[1]))
Y_point = np.empty((Y_werte_elli.shape[0], Y_werte_elli.shape[1]))

for zaehler in range(n*(n-1)):
    for i in range(schritte1):
        X_point[zaehler, i] = X_werte_elli[zaehler, i, 0]
        Y_point[zaehler, i] = Y_werte_elli[zaehler, i, 0]

                
resultat1 = np.array(resultat1)
times1    = np.array(times1)
kontakte1 = np.array(kontakte1)

# This is to asign colors of 'plasma' to the ellipses.
Test = mp.colors.Normalize(0, n)
Farbe = mp.cm.ScalarMappable(Test, cmap='plasma')
farben = [Farbe.to_rgba(l) for l in range(n)]    # color of the starting position

Dmcx = np.empty((n, schritte1))
Dmcy = np.empty((n, schritte1))
for kk in range(n):
    for i in range(schritte1):
        Dmcx[kk, i] = resultat1[i, n + kk]
        Dmcy[kk, i] = resultat1[i, 2 *n + kk]

Po_lam = []
for kk in range(n):
    Po_lam.append(sm.lambdify(qL + pL, [me.dot(Po_list[kk].pos_from(O), uv) for uv in (N.x, N.y)]))
Pox = np.empty((n, schritte1, 1))
Poy = np.empty((n, schritte1, 1))

for kk in range(n):
    for i in range(schritte1):
            Pox[kk, i] = Po_lam[kk](*[resultat1[i, j] for j in range(resultat.shape[1])], *pL_vals)[0]
            Poy[kk, i] = Po_lam[kk](*[resultat1[i, j] for j in range(resultat.shape[1])], *pL_vals)[1]

# needed to give the picture the right size.
xmin = np.min(Dmcx)
xmax = np.max(Dmcx)
ymin = np.min(Dmcy)
ymax = np.max(Dmcy)

# Data to draw the uneven street
cc = max(a1, b1)
strassex = np.linspace(xmin - 1.*cc, xmax + 1.*cc, schritte1)
strassey = [gesamt_lam(strassex[i], amplitude1, frequenz1) for i in range(schritte1)]
ymin = np.min(strassey)

def animate_pendulum(times1, Dmcx, Dmcy, Pox, Poy ):
    
    fig, ax = plt.subplots(figsize=(8, 8), subplot_kw={'aspect': 'equal'})
    
    ax.axis('on')
    ax.set_xlim(xmin - 1.*cc, xmax + 1.*cc)
    ax.set_ylim(ymin - 1., ymax + 1.*cc)
    ax.plot(strassex, strassey)
    
#    ax.plot(test_np, test1_np, color='green')
    LINE1 = ['line1' + str(i) for i in range(n)]
    LINE2 = ['line2' + str(i) for i in range(n)]
    LINE3 = ['line3' + str(i) for i in range(n)]
    LINE4 = ['line4' + str(i) for i in range(n)]
    LINE5 = ['line5' + str(i) for i in range(n)]

    for kk in range(n):
        line1, = ax.plot([], [], lw=0.5, color='yellow', marker='o', markeredgecolor='black')                      # center of the ellipse
        line2, = ax.plot([], [], 'o', color="white", markeredgecolor='black')                                # particle on the ellipse
        line3  = ax.axvline(kontakte1[kk, 0], linestyle='--', color=farben[kk])
        line4  = ax.axhline(gesamt_lam(kontakte1[kk, 0], amplitude1, frequenz1), linestyle='--', color=farben[kk])
        LINE1[kk] = line1
        LINE2[kk] = line2
        LINE3[kk] = line3
        LINE4[kk] = line4
    
        elli = patches.Ellipse((Dmcx[kk, 0], Dmcy[kk, 0]), width=2.*a1, height=2.*b1, angle=np.rad2deg(resultat1[0, kk]), zorder=1, fill=True, color=farben[kk], ec='black')
        LINE5[kk] = ax.add_patch(elli)

    LINE6 = [f'line6{i}{j}' for i, j in itt.permutations(range(n), r=2)]
    LINE7 = [f'line7{i}{j}' for i, j in itt.permutations(range(n), r=2)]
    
    zaehler = -1
    for ii, jj in itt.permutations(range(n), r=2):
        zaehler += 1
        line6, = ax.plot([], [], linestyle = '-', lw=0.5, color=farben[ii])
        line7, = ax.plot([], [], marker = 'o', color=farben[ii], markeredgecolor='white')

        LINE6[zaehler] = line6
        LINE7[zaehler] = line7


    def animate(i):
        message = (f'Running time {times1[i]:.2f} sec \n The white dot is the particle \n The doted lines of the same color cross at the (potential) impact points \n The lines connecting the ellipses give their potential impact points.')
        ax.set_title(message, fontsize=12)
        ax.set_xlabel('X direction', fontsize=12)
        ax.set_ylabel('Y direction', fontsize=12)

        for kk in range(n):
            LINE5[kk].set_center((Dmcx[kk, i], Dmcy[kk, i]))
            LINE5[kk].set_angle(np.rad2deg(resultat1[i, kk]))
                       
            LINE1[kk].set_data([Dmcx[kk, i]], [Dmcy[kk, i]])                  
            LINE2[kk].set_data(Pox[kk, i], Poy[kk ,i])      
            LINE3[kk].set_xdata([kontakte1[kk, i], kontakte1[kk, i]])
            wert = gesamt_lam(kontakte1[kk, i], amplitude1, frequenz1)
            LINE4[kk].set_ydata([wert, wert])

        for zaehler in range(n*(n-1)):
            LINE6[zaehler].set_data(X_werte_elli[zaehler, i], Y_werte_elli[zaehler, i])
            LINE7[zaehler].set_data([X_point[zaehler, i]], [Y_point[zaehler, i]])
                  
        return LINE1 + LINE2 + LINE3 + LINE4 + LINE5 + LINE6

    anim = animation.FuncAnimation(fig, animate, frames=schritte1,
                                   interval=2000*np.max(times1) / schritte1,
                                   blit=True)
    plt.close(fig)
    return anim

anim = animate_pendulum(times1, Dmcx, Dmcy, Pox, Poy)
print(f'it took {time.time() - start :.3f} sec to run the program BEFORE HTML')
HTML(anim.to_jshtml())    # needed, when run on an iPad, I know no other way to do it. It is SLOW!